## Mini-Capstone Project: Whale Activity vs. Price Movement, combining Coingecko and Dune Sim API data:

### Project Title: “Do Whales Move the Market?” — Analyzing Large Transfers vs Price Movements

### Project Goal: 
Analyze whether spikes in whale activity (large transfers) precede or correlate with significant price movements of a selected token.

### Core Components:
- Data Source
- Purpose
- Coingecko API
- Daily/hourly price data
- Dune Sim API
- Whale transactions (>$100K)
- Pandas, Matplotlib
- Data wrangling and visualization

### Skills Covered
- REST API requests (requests)

- JSON parsing

- Pandas DataFrame operations (merge, groupby, fillna)

- Time-series analysis

- Dual-axis plotting in Matplotlib and Seaborn

- Logic building and insight generation

## 1. Import Modules 

In [1]:
import os 
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from dotenv import load_dotenv
from dune_client.types import QueryParameter
from dune_client.client import DuneClient
from dune_client.query import QueryBase


## 2. API Setup 

In [3]:
# Load environment variables
load_dotenv()

# Get Dune and CoinGecko API keys from environment variables
DUNE_API_KEY = os.getenv("DUNE_API_KEY")
COINGECKO_API_KEY = os.getenv("COINGECKO_API_KEY")

# Get the Dune API client
dune_client = DuneClient(api_key=DUNE_API_KEY)

# Declare project parameters 
TOKEN_ID = "arbitrum"
VS_CURRENCY = "usd"
DAYS = 30
DUNE_QUERY_ID = 5614775 
WHALE_THRESHOLD_USD = 100000

if not os.path.exists("data"):
    os.makedirs("data")



## 3. Get CoinGecko Data (Arbitrum's Daily price Data for 30 days)

In [ ]:
def fetch_coingecko_data(token_id, vs_currency, days):
    file_path = f"data/arbitrum_historical_data.csv"

    # Check if cached
    if os.path.exists(file_path):
        price_df = pd.read_csv(file_path, parse_dates=["date"])
        if not price_df.empty and price_df["date"].max() >= (datetime.now() - pd.Timedelta(days=days)):
            print(f"Using cached data from {file_path}!!!")
            return price_df

    print(f"Fetching data from CoinGecko for arbitrum...")
    url = f"https://api.coingecko.com/api/v3/coins/arbitrum/market_chart"
    params = {
        "vs_currency": vs_currency,
        "days": days,
        "interval": "daily"
    }
    headers = {
        "Accept": "application/json",
        "X-CoinGecko-Api-Key": COINGECKO_API_KEY
    }

    response = requests.get(url, headers=headers, params=params)
    response.raise_for_status()
    data = response.json()

    price_df = pd.DataFrame(data["prices"], columns=["timestamp", "price"])
    price_df["date"] = pd.to_datetime(price_df["timestamp"], unit="ms")
    price_df = price_df[["date", "price"]]

    os.makedirs("data", exist_ok=True)
    price_df.to_csv(file_path, index=False)
    print(f"Saved to {file_path}")

    return price_df

if __name__ == "__main__":
    fetch_coingecko_data(TOKEN_ID, VS_CURRENCY, DAYS) 


Fetching data from CoinGecko for arbitrum...
Saved to data/arbitrum_historical_data.csv


## 4. Get Arbitrum Whale Transfers for 30 days 

In [6]:

def fetch_whale_transfers(query_id, token_id, freshness_days=1):
    file_path = f"data/arbitrum_whale_transfers.csv"

    # Try cache first
    if os.path.exists(file_path):
        dune_df = pd.read_csv(file_path, parse_dates=["date"])
        if not dune_df.empty and dune_df["date"].max() >= (datetime.now() - pd.Timedelta(days=freshness_days)):
            print(f"Using cached data from {file_path}!!!")
            return dune_df

    print(f"Checking latest execution for query 5614775...")
    latest_execs = dune_client.list_executions(query_id)
    if latest_execs and latest_execs[0].state == "QUERY_STATE_COMPLETED":
        exec_id = latest_execs[0].execution_id
        exec_time = latest_execs[0].execution_ended_at

        if exec_time and exec_time >= datetime.utcnow() - timedelta(days=freshness_days):
            print(f"Using existing execution {exec_id} from {exec_time}...")
            result = dune_client.get_execution(exec_id)
            dune_df = pd.DataFrame(result.data)
            dune_df["date"] = pd.to_datetime(dune_df["time"], unit="s", errors="coerce")
            dune_df.drop(columns=["time"], inplace=True, errors="ignore")
            os.makedirs("data", exist_ok=True)
            dune_df.to_csv(file_path, index=False)
            return dune_df

    # Run query if no recent execution
    print(f"No fresh execution found. Running query 5614775...")
    query = QueryBase(query_id=query_id)
    result = dune_client.run_query(query)
    dune_df = pd.DataFrame(result.data)
    dune_df["date"] = pd.to_datetime(dune_df["time"], unit="s", errors="coerce")
    dune_df.drop(columns=["time"], inplace=True, errors="ignore")
    os.makedirs("data", exist_ok=True)
    dune_df.to_csv(file_path, index=False)
    return dune_df

if __name__ == "__main__":
    fetch_whale_transfers(DUNE_QUERY_ID, TOKEN_ID, freshness_days=1)


Checking latest execution for query 5614775...


AttributeError: 'DuneClient' object has no attribute 'list_executions'